In [ ]:
# Downgrade sqlalchemy to a version less than 2.0
!pip install --upgrade 'sqlalchemy<2.0'

In [28]:
# imports
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
pd.options.display.max_rows = 10000

In [4]:
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
basics = pd.read_csv(basics_url, sep="\t", low_memory=False)

In [6]:
Akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
Akas = pd.read_csv(Akas_url, sep="\t", low_memory=False)

In [7]:
Rating_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
Rating = pd.read_csv(Rating_url, sep="\t", low_memory=False)

In [8]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [9]:
Akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [10]:
Rating.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2003
1,tt0000002,5.8,269
2,tt0000003,6.5,1896
3,tt0000004,5.5,178
4,tt0000005,6.2,2679


**Replacing \N by nan per data dictionary**

In [14]:
basics.replace({'\\N':np.nan},inplace=True)

In [15]:
Akas.replace({'\\N':np.nan},inplace=True)

In [16]:
Rating.replace({'\\N':np.nan},inplace=True)

**Cleaning Basics database**

In [17]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10250934 entries, 0 to 10250933
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 703.9+ MB


In [88]:
#Eliminate movies that are null for runtimeMinutes
basics.dropna(subset=['runtimeMinutes'], inplace=True)

In [90]:
basics['runtimeMinutes'].isna().sum()

0

In [91]:
#Eliminate movies that are null for genres
basics.dropna(subset=['genres'], inplace=True)

In [92]:
basics['genres'].isnull().sum()

0

In [ ]:
basics['runtimeMinutes'].value_counts()

In [ ]:
basics['genres'].value_counts()

In [95]:
#keep only titleType==Movie
df1= basics[basics['titleType']=="movie"]

In [96]:
basics['titleType'].value_counts()

tvEpisode       1584837
short            617288
movie            390879
video            185741
tvMovie           92916
tvSeries          92463
tvSpecial         19272
tvMiniSeries      18092
tvShort            8752
videoGame           339
Name: titleType, dtype: int64

In [97]:
len(df1.index)


390879

In [98]:
#keep startYear 2000-2022
df2=df1[(df1["startYear"]=="2000") | (df1["startYear"]=="2021") | (df1["startYear"]=="2022")]

In [99]:
df2.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13081,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,94,Documentary
76051,tt0077684,movie,Histórias de Combóios em Portugal,Histórias de Combóios em Portugal,0,2022,NaN,46,Documentary
81471,tt0083283,movie,Victor Seastrom,Victor Sjöström - ett porträtt av Gösta Werner,0,2021,NaN,65,"Biography,Documentary"
94102,tt0096235,movie,Taxi Killer,Taxi Killer,0,2022,NaN,106,"Action,Crime,Drama"
100065,tt0102362,movie,Istota,Istota,0,2000,NaN,80,"Drama,Romance"


**Cleaning akas database**

In [ ]:
akas= akas[akas['titleType']=="movie"]

In [ ]:
# Exclude movies that are included in the documentary category.
is_documentary = df2['genres'].str.contains('Documentary',case=False)
df3 = df2[~is_documentary]

In [ ]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
basics = basics[keepers]
